TP1

Laboratorio de Datos

1er Cuatrimestre - 2024

Iván Exequiel Pintos - Joaquín Rovner - Juan José García Vizioli

- Importamos todas las librerías que venimos usando:

In [ ]:
import numpy as np

import pandas as pd

import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score    # Medidas de desempeño

from formulaic import model_matrix
from formulaic import Formula

- Leemos los dataset:

In [ ]:
sube_2023_file_path = '../LDD-TP1/sube-2023.csv'
sube_2023_regresion_file_path = '../LDD-TP1/sube-2023-regresion.csv'

SUBE_2023 = pd.read_csv(sube_2023_file_path)
SUBE_2023_regresion = pd.read_csv(sube_2023_regresion_file_path)

#Procesamiento de Datos

##1)

###a)

Visualizamos el tipo de dato de cada columna:

In [ ]:
#Definimos una función que nos permita ver el tipo de dato correspondiente a cada columna del DataFrame.

def ver_tipos_datos_columnas(df):
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)
    lista = []
    for col in df:
        lista.append([col,type(df[col][0])])
    display(pd.DataFrame(lista, columns = ["COLUMNA","TIPO DE DATO"]))

ver_tipos_datos_columnas(SUBE_2023)

Luego, le cambiamos el formato a la columna DIA_TRANSPORTE:

In [ ]:
SUBE_2023['DIA_TRANSPORTE'] = pd.to_datetime(SUBE_2023['DIA_TRANSPORTE']) #investigar argumento "format"

Corroboramos que el tipo de dato de la columna ahora es un time_stamp:

In [ ]:
ver_tipos_datos_columnas(SUBE_2023)

###b)

####i)

Agregamos la columna FECHA_DIA a partir de DIA_TRANSPORTE

In [ ]:
SUBE_2023["FECHA_DIA"] = SUBE_2023['DIA_TRANSPORTE'].apply(lambda x : x.strftime('%a'))

Vemos que la columna sigue el formato de 3 letras en inglés pedido:

In [ ]:
SUBE_2023.head()

####ii)


Agregamos la columna FECHA_ORDINAL a partir de DIA_TRANSPORTE, que numera los días desde 1 hasta 365.

In [ ]:
SUBE_2023["FECHA_ORDINAL"] = SUBE_2023['DIA_TRANSPORTE'].apply(lambda x : int(x.strftime('%-j')))

Vemos que la columna FECHA_ORDINAL tiene el formato pedido y que además sus datos son del tipo int:

In [ ]:
display(SUBE_2023.head())
ver_tipos_datos_columnas(SUBE_2023["FECHA_ORDINAL"])

####iii)

Agregamos la columna FECHA_MES a partir de DIA_TRANSPORTE

In [ ]:
SUBE_2023["FECHA_MES"] = SUBE_2023['DIA_TRANSPORTE'].apply(lambda x : x.strftime('%m'))

Vemos que la columna sigue el formato de string de número de 2 dígitos pedido

In [ ]:
SUBE_2023.head()

##2)

Creamos el DataFrame datos_amba, el cual solo contiene datos de AMBA y excluye los datos preliminares.

In [ ]:
datos_amba = SUBE_2023[(SUBE_2023['AMBA'] == 'SI') & (SUBE_2023['DATO_PRELIMINAR'] == 'NO')]

columnas_datos_amba = ['DIA_TRANSPORTE','FECHA_DIA','FECHA_MES','FECHA_ORDINAL','JURISDICCION','LINEA','CANTIDAD','TIPO_TRANSPORTE']

datos_amba = (datos_amba[columnas_datos_amba].rename(columns = {'DIA_TRANSPORTE' : 'FECHA', 'CANTIDAD' : 'PASAJEROS'})
            .rename(columns=str.lower).reset_index().drop(["index"], axis = 1)) #transformamos en minúsculas y reseteamos índices.

datos_amba.head()

##3)

###a)

Identificamos la proporción de la cantidad total anual de pasajeros que le corresponde a cada medio de transporte

In [ ]:
datos_amba['tipo_transporte'].unique() #Ver qué tipos de transporte hay

In [ ]:
total_pasajeros_amba = datos_amba['pasajeros'].sum()

datos_amba_colectivo = datos_amba[datos_amba['tipo_transporte'] == 'COLECTIVO']
datos_amba_tren = datos_amba[datos_amba['tipo_transporte'] == 'TREN']
datos_amba_subte = datos_amba[datos_amba['tipo_transporte'] == 'SUBTE']

pasajeros_colectivo_amba = datos_amba_colectivo['pasajeros'].sum()
pasajeros_tren_amba = datos_amba_tren['pasajeros'].sum()
pasajeros_subte_amba = datos_amba_subte['pasajeros'].sum()

porcentaje_pasajeros_colectivo = pasajeros_colectivo_amba * 100 / total_pasajeros_amba
porcentaje_pasajeros_tren = pasajeros_tren_amba * 100 / total_pasajeros_amba
porcentaje_pasajeros_subte = pasajeros_subte_amba * 100 / total_pasajeros_amba

print('\033[1mPorcentaje anual en colectivo:\033[0m', str(round(porcentaje_pasajeros_colectivo, 2)) + '%')
print('\033[1mPorcentaje anual en tren:\033[0m', str(round(porcentaje_pasajeros_tren, 2)) + '%')
print('\033[1mPorcentaje anual en subte:\033[0m', str(round(porcentaje_pasajeros_subte, 2)) + '%')

###b)

Identificamos la tupla (mes, línea de subte) donde viajó la mayor cantidad de pasajeros

In [ ]:
fila_record_subte_amba = datos_amba_subte.nlargest(1, 'pasajeros').iloc[0]

record_subte = mes_record_subte, linea_record_subte = fila_record_subte_amba['fecha_mes'], fila_record_subte_amba['linea']

print('\033[1mMes y linea con mayor cantidad de pasajeros en subte:\033[0m', record_subte)

###c)

Identificamos el día hábil con menor desvío estándar en cantidad de pasajeros

In [ ]:
dia_habil_menor_std = datos_amba.groupby('fecha_dia')['pasajeros'].std().drop(['Sun','Sat']).idxmin()

dias_completos_esp = {
    'Mon': 'Lunes',
    'Tue': 'Martes',
    'Wed': 'Miércoles',
    'Thu': 'Jueves',
    'Fri': 'Viernes',
    'Sat': 'Sábado',
    'Sun': 'Domingo'
}

print('\033[1mDía hábil con menor desvío estándar en cantidad de pasajeros:\033[0m',dias_completos_esp[dia_habil_menor_std])

#Análisis Exploratorio

##4)

In [ ]:
#Ocultamos las advertencias para una visualización más clara y limpia de los gráficos

import warnings
warnings.filterwarnings('ignore')

En primer lugar, observamos la proporción de pasajeros transportados en el AMBA vs el resto del país.

In [ ]:
df_agrupado = SUBE_2023.groupby('AMBA')['CANTIDAD'].sum().reset_index()
(
    so.Plot(data = df_agrupado, x = "AMBA", y = "CANTIDAD")
    .add(so.Bar())
)

Vemos que efectivamente la mayor cantidad de pasajeros transportados se concentra en el AMBA.
Por lo tanto, nos concentraremos en comparar el comportamiento de los distintos tipos de transporte del AMBA a lo largo de 2023.

### Evolución de los pasajeros transportados en SUBTE a lo largo del año 2023

In [ ]:
#Creamos un nuevo DataFrame a partir de datos_amba para estudiar el comportamiento de los subtes.

subtes = datos_amba[datos_amba["tipo_transporte"]=="SUBTE"].reset_index().drop(["index"], axis = 1)
subtes = subtes[subtes["linea"] != "LIN_PREMETRO"] #Excluímos el premetro por la escases de datos en comparación con las otras líneas.
subtes["linea"].replace({"LINEA_A": "LINEA SUBTE A", "LINEA_B": "LINEA SUBTE B"}, inplace = True) #Normalizamos los nombres de las columnas.

#Modelado

## Ejercicio 5)

### a)

Visualizamos el dataset a utilizar

In [ ]:
SUBE_2023_regresion.head()

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO


Completamos el código filtrando la base de datos con las 4 condiciones pedidas. Además, reemplazamos los espacios por guiones bajos en los nombres de las columnas para evitar eventuales problemas con formulaic.

In [ ]:
condicion = ((SUBE_2023_regresion.AMBA == 'SI') & (SUBE_2023_regresion.TIPO_TRANSPORTE == 'COLECTIVO')) & ((SUBE_2023_regresion.JURISDICCION == 'NACIONAL') & (SUBE_2023_regresion.PROVINCIA == 'JN'))
datos_ColectivoJN = SUBE_2023_regresion[condicion]

cols = datos_ColectivoJN.LINEA.unique()  # Los nombres de las lineas de colectivo

pasajeros_por_linea = pd.DataFrame()

for col in cols:
    datos_linea = datos_ColectivoJN[datos_ColectivoJN.LINEA == col][["DIA_TRANSPORTE", "CANTIDAD"]]
    datos_linea = datos_linea.set_index("DIA_TRANSPORTE").rename(columns={"CANTIDAD": col})
    pasajeros_por_linea = pd.concat([pasajeros_por_linea, datos_linea], axis=1)

pasajeros_por_linea.columns = [col.replace(' ', '_') for col in pasajeros_por_linea.columns]
display(pasajeros_por_linea.head())


,BS_ASLINEA_123,BSAS_LINEA_002,BSAS_LINEA_009,BSAS_LINEA_010,BSAS_LINEA_015,BSAS_LINEA_017,BSAS_LINEA_019,BSAS_LINEA_020,BSAS_LINEA_021,BSAS_LINEA_022,...,LINEA_5,LINEA_50,LINEA_6,LINEA_7,LINEA_76,LINEA_8,LINEA_099,LINEA_119_AMBA,LINEA_164_AMBA,LINEA_119
DIA_TRANSPORTE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,1681,5670,5644,5177,9109,7013,1604,2617,11235,2202,...,4832.0,4839,2449,2184,3797,5317,1882,1573.0,3210,NaN
2023-01-02,9440,21759,23284,21176,45269,29962,13921,11934,55611,13823,...,20333.0,16221,10486,12774,17374,23250,10322,7345.0,19362,NaN
2023-01-04,10540,24755,25405,23806,52873,33659,16172,13366,61721,16206,...,23041.0,18448,12193,13949,19860,26294,11981,7925.0,21784,NaN
2023-01-05,10408,25772,26489,24688,53436,35182,16569,12929,62510,16863,...,23928.0,18481,12888,14668,21235,27216,11787,7597.0,22972,NaN
2023-01-06,10530,26021,26458,24167,53163,35212,16459,12955,63528,16846,...,24247.0,19300,12703,14199,23472,27073,11567,7811.0,23411,NaN


### b)

Corroboramos que al hacer .dropna() en el eje correspondiente a las columnas (axis = 1) perdemos 12 columnas:

In [ ]:
pasajeros_por_linea = pasajeros_por_linea.dropna(axis = 1)
display(pasajeros_por_linea.head())


,BS_ASLINEA_123,BSAS_LINEA_002,BSAS_LINEA_009,BSAS_LINEA_010,BSAS_LINEA_015,BSAS_LINEA_017,BSAS_LINEA_019,BSAS_LINEA_020,BSAS_LINEA_021,BSAS_LINEA_022,...,LINEA_4,LINEA_42,LINEA_44,LINEA_50,LINEA_6,LINEA_7,LINEA_76,LINEA_8,LINEA_099,LINEA_164_AMBA
DIA_TRANSPORTE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,1681,5670,5644,5177,9109,7013,1604,2617,11235,2202,...,2942,4029,4649,4839,2449,2184,3797,5317,1882,3210
2023-01-02,9440,21759,23284,21176,45269,29962,13921,11934,55611,13823,...,15329,19072,20387,16221,10486,12774,17374,23250,10322,19362
2023-01-04,10540,24755,25405,23806,52873,33659,16172,13366,61721,16206,...,17836,21609,23595,18448,12193,13949,19860,26294,11981,21784
2023-01-05,10408,25772,26489,24688,53436,35182,16569,12929,62510,16863,...,18044,21993,23687,18481,12888,14668,21235,27216,11787,22972
2023-01-06,10530,26021,26458,24167,53163,35212,16459,12955,63528,16846,...,18122,22421,23731,19300,12703,14199,23472,27073,11567,23411


### c)

Seleccionamos 3 sets de 5 columnas cada uno con los siguientes criterios, todos en base al dataframe pasajeros_por_linea:


*   Set 1: son las 5 columnas que mayor correlación tengan con BSAS_LINEA_009. La correlación entre columnas se calcula según el coeficiente R de Pearson.
  
*   Set 2: son las 5 columnas que tienen menor desviación estándar en su diferencia respecto de BSAS_LINEA_009. Se calcula un vector de diferencias (que pueden ser negativas) de cada columna contra BSAS_LINEA_009, y se eligen las 5 que tengan menor desviación en este vector (lo que se puede interpretar como que son las 5 que mejor siguen la "forma" de la curva BSAS_LINEA_009, aunque pueda haber una constante entre ellas y BSAS_LINEA_009)

*   Set 3: son las 5 columnas con menor promedio en su vector de distancias (igual que el vector de diferencias, pero en valor absoluto) respecto de BSAS_LINEA_009. Estas representan las 5 columnas que más cerca están de BSAS_LINEA_009 en todo momento.




In [ ]:
distancias = pasajeros_por_linea.copy().drop(columns = 'BSAS_LINEA_009')
diferencias = distancias.copy()

for column in pasajeros_por_linea.columns:
    if column != 'BSAS_LINEA_009':
        diferencias[column] = pasajeros_por_linea[column] - pasajeros_por_linea['BSAS_LINEA_009']
        distancias[column] = abs(diferencias[column])

print('Vectores de distancias:')
display(distancias.head())
print('Vectores de diferencias:')
display(diferencias.head())

Vectores de distancias:


,BS_ASLINEA_123,BSAS_LINEA_002,BSAS_LINEA_010,BSAS_LINEA_015,BSAS_LINEA_017,BSAS_LINEA_019,BSAS_LINEA_020,BSAS_LINEA_021,BSAS_LINEA_022,BSAS_LINEA_024,...,LINEA_4,LINEA_42,LINEA_44,LINEA_50,LINEA_6,LINEA_7,LINEA_76,LINEA_8,LINEA_099,LINEA_164_AMBA
DIA_TRANSPORTE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,3963,26,467,3465,1369,4040,3027,5591,3442,1955,...,2702,1615,995,805,3195,3460,1847,327,3762,2434
2023-01-02,13844,1525,2108,21985,6678,9363,11350,32327,9461,7113,...,7955,4212,2897,7063,12798,10510,5910,34,12962,3922
2023-01-04,14865,650,1599,27468,8254,9233,12039,36316,9199,8162,...,7569,3796,1810,6957,13212,11456,5545,889,13424,3621
2023-01-05,16081,717,1801,26947,8693,9920,13560,36021,9626,7863,...,8445,4496,2802,8008,13601,11821,5254,727,14702,3517
2023-01-06,15928,437,2291,26705,8754,9999,13503,37070,9612,8416,...,8336,4037,2727,7158,13755,12259,2986,615,14891,3047


Vectores de diferencias:


,BS_ASLINEA_123,BSAS_LINEA_002,BSAS_LINEA_010,BSAS_LINEA_015,BSAS_LINEA_017,BSAS_LINEA_019,BSAS_LINEA_020,BSAS_LINEA_021,BSAS_LINEA_022,BSAS_LINEA_024,...,LINEA_4,LINEA_42,LINEA_44,LINEA_50,LINEA_6,LINEA_7,LINEA_76,LINEA_8,LINEA_099,LINEA_164_AMBA
DIA_TRANSPORTE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,-3963,26,-467,3465,1369,-4040,-3027,5591,-3442,1955,...,-2702,-1615,-995,-805,-3195,-3460,-1847,-327,-3762,-2434
2023-01-02,-13844,-1525,-2108,21985,6678,-9363,-11350,32327,-9461,7113,...,-7955,-4212,-2897,-7063,-12798,-10510,-5910,-34,-12962,-3922
2023-01-04,-14865,-650,-1599,27468,8254,-9233,-12039,36316,-9199,8162,...,-7569,-3796,-1810,-6957,-13212,-11456,-5545,889,-13424,-3621
2023-01-05,-16081,-717,-1801,26947,8693,-9920,-13560,36021,-9626,7863,...,-8445,-4496,-2802,-8008,-13601,-11821,-5254,727,-14702,-3517
2023-01-06,-15928,-437,-2291,26705,8754,-9999,-13503,37070,-9612,8416,...,-8336,-4037,-2727,-7158,-13755,-12259,-2986,615,-14891,-3047


In [ ]:
from scipy.stats import pearsonr

correlaciones = pd.Series(index=distancias.columns)
promedios = correlaciones.copy()
desviaciones = correlaciones.copy()

for linea in distancias.columns:
  correlaciones[linea] =  abs(pearsonr(pasajeros_por_linea['BSAS_LINEA_009'], pasajeros_por_linea[linea])[0])
  promedios[linea] = distancias[linea].mean()
  desviaciones[linea] = diferencias[linea].std()

Finalmente los 3 sets seleccionados quedan así:

In [ ]:
n = 5
set_1 = list(correlaciones.nlargest(n).index)
set_2 = list(desviaciones.nsmallest(n).index)
set_3 = list(promedios.nsmallest(n).index)

print(set_1, set_2, set_3, sep = '\n')

['LINEA_101', 'BSAS_LINEA_146', 'LINEA_7', 'BSAS_LINEA_024', 'LINEA_107']
['BSAS_LINEA_146', 'BSAS_LINEA_100', 'LINEA_107', 'BSAS_LINEA_117', 'BSAS_LINEA_188']
['BSAS_LINEA_146', 'BSAS_LINEA_169', 'BSAS_LINEA_078', 'BSAS_LINEA_045', 'LINEA_164_AMBA']


Para cada set de datos se proponen las siguientes formulas:

*    Formula 1: modelo lineal respecto de las columnas del Set 1, sin interacciones entre ellas. No hay ordenada al origen para simplificar el modelo.
*    Formula 2: igual que la Formula 1, pero con las columnas del Set 2 y esta vez con ordenada al origen (ya que estas columnas solo siguen la forma de la curva de BSAS_LINEA_009, y bien podrían necesitar una constante para acercarse a ella).
*    Formula 3: todas las interacciones posibles entre las columnas del Set 3.

In [ ]:
formula_1 = '+'.join(set_1) + '-1'
formula_2 = '+'.join(set_2)
formula_3 = '*'.join(set_3)

formulas = [formula_1, formula_2, formula_3]

### d)

No se usó regresión Ridge para ningún modelo.

### e)

Primeramente, se separó a los datos de pasajeros_por_linea en 2 dataframes: un 20% aleatorio se reservó para un testeo final en pasajeros_por_linea_test y el 80% restante se usó para entrenar y validar en pasajeros_por_linea_no_test.



De los datos de no_test, se armaron 11 (número elegido arbitrariamente) pliegos aleatorios. En cada pliego, se calculó el desempeño de cada modelo según el parámetro de bondad R^2.

Luego, se tomaron en consideración dos medidas:
por un lado, cuál fue el modelo con mejor desempeño en cada pliego; por otro, el R^2 promedio (a lo largo de todos los pliegos) de cada modelo.

In [ ]:
from scipy import stats


pasajeros_por_linea_no_test, pasajeros_por_linea_test  = train_test_split(pasajeros_por_linea, test_size=0.2, random_state=76)

cv = KFold(n_splits=11, random_state=42, shuffle=True)
pliegos = cv.split(pasajeros_por_linea_no_test)

modelos_mejores_rcm = []

rcms = [[], [], []]

for train_indexes, valid_indexes in pliegos:
  mejor_rcm = np.infty
  ganador = None
  for i, formula in enumerate( formulas):
    y, X = (
        Formula('BSAS_LINEA_009 ~ ' + formula)
        .get_model_matrix(pasajeros_por_linea_no_test)
    )

    X_train, X_valid, y_train, y_valid = X.iloc[train_indexes], X.iloc[valid_indexes], y.iloc[train_indexes], y.iloc[valid_indexes]


    modelo = LinearRegression(fit_intercept = False)
    modelo.fit(X_train, y_train)

    y_pred = modelo.predict(X_valid)

    r2 = r2_score(y_valid, y_pred)
    rcms[i].append(r2)

    if r2 < mejor_rcm:
      mejor_rcm = r2
      ganador = i+1

  modelos_mejores_rcm.append(ganador)

rcms_promedios = [np.mean(r) for r in rcms]
print('R2 promedio de cada modelo: ', rcms_promedios)

print('Modelo ganador de cada pliego: ', modelos_mejores_rcm)
print('Modelo que ganó más veces: ', stats.mode(modelos_mejores_rcm)[0])

R2 promedio de cada modelo:  [0.9491399046116975, 0.9451314906984666, 0.28936927875466734]
Modelo ganador de cada pliego:  [1, 1, 2, 2, 3, 3, 3, 1, 3, 2, 2]
Modelo que ganó más veces:  2


Se ve que la Fórmula 1 tuvo en promedio un mejor rendimiento que las otras, pero al mirar cada pliego individualmente, en total la Fórmula 2 ganó más veces. Ambos promedios son similares, y la Fórmula 2 solo tuvo una victoria extra por encima de la Fórmula 1. En principio, según el criterio preferido, podría elegirse cualquiera de los dos como el ganador.

En este caso, puesto que el promedio es un poco más sensible a outliers, se tomó a la Fórmula 2 como la ganadora.

La Fórmula 3 queda totalmente descartada. Si bien tuvo la misma cantidad de victorias que la Fórmula 1, su promedio es bajísimo. Esto quiere decir que hubo al menos un pliego donde su rendimiento fue tal vez incluso peor que un modelo constante.

Al entrenar la Fórmula 2 con todos los datos de pasajeros_por_linea_no_test, se ve que tiene un buen rendimiento con el 20% de los datos reservados para testeo, con un R^2 de 0,961.

In [ ]:
formula = 'BSAS_LINEA_009 ~ ' + formula_2

y_no_test, X_no_test = (
    Formula(formula)
    .get_model_matrix(pasajeros_por_linea_no_test)
)


modelo_no_test = LinearRegression(fit_intercept = False)
modelo_no_test.fit(X_no_test, y_no_test)

y_test, X_test = (
    Formula(formula)
    .get_model_matrix(pasajeros_por_linea_test)
)

y_pred = modelo_no_test.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(r2)

0.9609590506842676


(En este caso en particular, si se hubiera elegido la Fórmula 1, se habría tenido un R^2 aun mejor en los datos de testeo, lo cual se puede comprobar fácilmente editando el primer renglon de la celda de código de arriba. Como sea, esto sería fácilmente explicable por fluctuaciones estadísticas, y la idea no es tomar decisiones luego de haber mirado los datos de testeo, por lo que la Fórmula 2 sigue siendo la elegida).

### f y g)

Como los modelos van a ser eventualmente contrastados con otros datos de testeo desconocidos, no hay nada que impida en este punto usar todo el dataset disponible para entrenar el modelo final reportado. Usando todos los datos posibles (incluyendo los que anteriormente eran solo de testeo), queda finalmente el modelo siguiente:

In [ ]:
y_final, X_final = (
    Formula(formula)
    .get_model_matrix(pasajeros_por_linea)
)

modelo_final = LinearRegression(fit_intercept = False)
modelo_final.fit(X_final, y_final)

formula_final = 'BSAS_LINEA_009 = ' + str(round(modelo_final.coef_[0][0],3)) + '+' + '+'.join([str(round(c,3)) + '*' + l for c, l in zip(modelo_final.coef_[0][1:],X_final.columns[1:])])
print('Modelo final:', formula_final, sep = '\n')


Modelo final:
BSAS_LINEA_009 = -1142.563+0.553*BSAS_LINEA_146+0.162*BSAS_LINEA_100+0.178*LINEA_107+0.304*BSAS_LINEA_117+-0.225*BSAS_LINEA_188
